#### 📚PN : 2
📚[PN : 1](https://www.kaggle.com/sarabhian/gbr-extremely-beginner-level-guide-1)

# 🍀WELCOME To My Notebook 
This is my 2022's first Notebook... 

**HAPPY NEW YEAR TO ALL** 🎉🎉🎊


I will try to explore and explain the notebook as friendly as I can, I can guarantee  you that it will be very interesting  to read this notebook 😂

# 🔅 Basic EDA

**Understanding The Problem** all of us have read the description of problem statement and it says ,
>There are two (fictitious) independent store chains selling Kaggle merchandise that want to become the official outlet for all things Kaggle. We've decided to see if the Kaggle community could help us figure out which of the store chains would have the best sales going forward. So, we've collected some data and are asking you to build `forecasting models` to help us decide.

As soon as you hear the name `time series` or`forecasting model` then change your attitude towards that problem , because this problem need to be handled in very unique manner, we can't apply basic supervised learning models directly to predict the future,

let's understand what time series means :

**Time Series Prediction**  
* The data which follows the periodicity with respect to time  is `Time Series Data` ,
* Generally Data looks like `Sine waves` or like `stock market curves` but repeating the curves after some interval
* We want to understant how the trends of predictor `y`  ( in our case Y is no. of sales ) changes with respect to time 
* and we want to predict the future trends of predictor `y` ( in simple we want to predict the future sales , future stock prices , future product evaluations and many more ...)


I tried to discuss few simple , easily understandable + effective techniques to solve this problem
let's dive into the data science of the future prediction 🏊‍♀️🏊‍♀️



## 💠 Importing libraries

In [ ]:
## basic libraries
import os
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
%matplotlib inline

## libraries for feature engineering
from sklearn.preprocessing import LabelEncoder


## 💠 importing DATA

In [ ]:
path = "../input/tabular-playground-series-jan-2022"
tdf = pd.read_csv(path + "/train.csv")
Tdf = pd.read_csv(path + "/test.csv")
sdf = pd.read_csv(path +"/sample_submission.csv")

In [ ]:
print(f"train:{tdf.shape} \ntest:{Tdf.shape}\ntest-train ratio: {(Tdf.shape[0])/(tdf.shape[0])}")

ok we have very less data , which is always a bad thing for us to create models 🤣

## 💠Determining the predictor Y and it's behaviour
let's find the label `Y` from the data and  see what exactly we have to predict

In [ ]:
tdf.head()

In [ ]:
Tdf.head()

So, here the feature named `num_sold` is the predictor `y` for this problem, let's see what this `num_sold` has inside it

In [ ]:
tdf.num_sold.describe()

In [ ]:
print("total predictions :", tdf.num_sold.count())
print("\ntotal unique predictions:",tdf.num_sold.nunique())
print("\nnull_predictions: " ,tdf.num_sold.isnull().sum())

In [ ]:
sns.lineplot(y=tdf['num_sold'],x=tdf['date'] ) # liquidity 
plt.show()
sns.histplot(data=tdf ,x= "num_sold" ) # distribution 
plt.show()
sns.boxplot(data=tdf, x= "num_sold") # variance

`fig 1` this fig is computationally heavy to plot as we have not seperated `date` with respect to specific `year` , `month` and `day` but for first glance we can see our predictor `y` is following periodicity which clearly indicates this could be the problem of time series prediction even if it is not specified in compitition

`fig2` 😮 chi-squared curve or skewed normal distribution... looks great curve, <br>
`Note:`  predictor `y` is distributed under chi_square distribution, which clearly indicates under normal conditions  we should be using categorical ML Models like multiclass  classification , clustering or softmax or svm  .....`but` this is not a supervised learning problem it's problem of future prediction which comes under `time series predictions`  so our entire methodology to solve the problem changes ...<br>

`fig3` it shows that predictor `y` is having some outliers which are indications of high sales on some festival or some special ocassion

# 🔅  Feature Engineering For Time Series Data 

In [ ]:
tdf.head(3)

In [ ]:
tdf.info()

soka 😮 ,as we can see there are `4 features` which are `objects`and `2` are `int64` , and if we see tdf.head output then we can see 3 of the 4 objects are `categorical:`  `country` , `store` ,`product` and 1 of them is `time` , time could be categorical , ordinal, continuous .. anything [refer here for understanding time data type](https://statisticalanalysisconsulting.com/is-time-nominal-ordinal-interval-or-ratio-is-it-categorical-or-continuous/) most of the time we drop the date in the datasets but in this problem  `date` is the main feature 

## ⭕ creating new time features

here we try to break the timeline into different parts  like `year , month , week , days , weekdays , quarter , Hours , Minutes , seconds` etc to understand how the predictor `y` ( no. of sales) is changing with respect to these different timelines


In [ ]:
def time_splitter(df,date): #here df = data and date= given time feature 
    df[date]         = pd.to_datetime(df[date])
    df['year']       = df[date].dt.year
    df['month']      = df[date].dt.month
    df['day']        = df[date].dt.day
    df['week']       = df[date].dt.isocalendar().week
    df['weekday']    = df[date].dt.weekday
    df['weekend']    = (df[date].dt.weekday>4).astype(int)
    df['day_of_yr']  = df[date].dt.dayofyear
    df['quarter']    = df[date].dt.quarter
    return df

# time splitting for training data
tdf=time_splitter(tdf,'date')
tdf.loc[300:310]

- - - - - -

In [ ]:
# time splitting for test data
Tdf=time_splitter(Tdf,'date')
Tdf.head()

In [ ]:
tdf=tdf.drop(['row_id','date'],axis=1)

In [ ]:

col=list(tdf.columns)  
col

### 💢null value analysis 

In [ ]:
print(f"values with None:\n{tdf[tdf.values==None].count()} \n\nvalue with null or empty:\
     \n{tdf.isnull().nunique()}")

for beginners above line of code may look difficult to understand 😅 
but it's simple 
1. first I used `f" {} {}"`  to write code in just one print function 
2. then to break the lines I used `\` at end of line after **value with null or empty:`\`** 👈
3. inside {} I used two functions 
   * `tdf[tdf.values==None].count()` comapres values of data with `None` data type since nunique can't detect this data type as null value 
   * `tdf.nunique(axis=0 , dropna =False)`  compares null values like `[] ," " , NaN ` etc 

> outputs for values with none ==0 => there is no None value<br>
> outputs for null values ==1 => isnull() writes true or false as nunique of isnull is 1 which means either all isnull values are True (entire data is Null)  or all isnull values are false ( there is no null value in the data)

so conclusion is there is no null value in this dataset everthing looks great and easy😂

In [ ]:
sns.set_style('darkgrid')
sns.pairplot(data=tdf ,height=5 , corner=True ,diag_kind='kde')

this above plot looks scary isn't it ? 👹👹 most of the people consider pairplot as nonsense plot because people won't understand it🤣🤣, but let me tell you the insights that I am getting from this plot
double click of graph and see each graph carefully to check the below given insights<br>
( considering `num_sold` =`ns` for decreasing tyoing load 😁)<br>
1. first column of plot:

 * `quarter` vs `ns` : in 2nd and 4th quarter we had to more sales
 * `day_of_year` vs `ns` : for days 50:150 and for days 350:365 we had high sales
 * `weekend` vs `ns` : we had little high sales on weekends (as expected)
 * `weekday` vs `ns` : --||--
 * `day` vs `ns` : at end of month we have high sales
 * `month` vs `ns` : 4th ,5th and 12th month had more sales
 * `year` vs `ns` : 2017 and 2018 had more sales ( I will say sales are increasing each year)
 * `index` vs `ns` : indicates num_sold is chi-square distributed which is expected curve for sales 
 <br><br>
 similarly I can tell each plots meaning but I am tired of writing so you guys decide on your own 😂

In [ ]:
cols = ['country','store','product']
[print(f"\nTotal Sales by {i}  :\n{tdf[i].value_counts()}") for i in cols]

total data is symmetric ... this is doing of the team who arranged this competition , we don't get symmetric data in general😉

##  💠 Handling Categorical features

since our data has huge classes we will use label encoding 

In [ ]:
from collections import defaultdict
d= defaultdict(LabelEncoder)         # trick to do encoding and decoding vary easily 😁
 
x = tdf[['country' ,'store','product','year']]

x=x.apply(lambda x: d[x.name].fit_transform(x))
x.head()
tdf[['country' ,'store','product','year']]=x
tdf

In [ ]:
# for inverting the x to it's original data if needed
x=x.apply(lambda x: d[x.name].inverse_transform(x))
x.head()

references [lambda](https://towardsdatascience.com/lambda-functions-with-practical-examples-in-python-45934f3653a8)
[LabelEncoder](https://scikit-learn.org/stable/modules/preprocessing_targets.html) [trick to use encoding](https://stackoverflow.com/questions/24458645/label-encoding-across-multiple-columns-in-scikit-learn)

ok now we got data all in numerical form let's do some more visualizations and analysis

### 💢outlier analysis

In [ ]:
  tdf.describe()

the only outlier here is `num_sold`  which we are not goint to fix as it will help us to underfit the data

let's do some interesting outlier analysis for this new year 😂💥

#### 1.Eliptic Envelope Algorithm for outlier detection

suppose we want to see how much % of data is outside the regionr as per our recommended outlier region we can use boxplot but there is something more good for it as follows

let's see we keep data with 98% inlier and 2 % outlier ==>  contamination = 0.02

In [ ]:
from sklearn.covariance import EllipticEnvelope
X= tdf["num_sold"].values
X=X.reshape((13149,2)) # 13149 = len(X)/2 since X need to be 2D array
# print(X)
envlp = EllipticEnvelope(contamination = 0.02 ,random_state=10)  # contamination == % of data outside ellipse
pred = envlp.fit_predict(X)
# Extract outliers
outlier_index = np.where(pred<=-1)
outlier_values = X[outlier_index]

# Plot the data
sns.set(rc={'figure.figsize':(20,12)})
sns.set_style('darkgrid')
sns.scatterplot(x=X[:,0], y=X[:,1] )
sns.scatterplot(x=outlier_values[:,0], 
                y=outlier_values[:,1], color='r')
plt.title("Elliptic Envelope Outlier Detection", fontsize=15, pad=15)

#plt.savefig("Elliptic Envelope Detection.png", dpi=80)

# outlier count 
print("total outliers" ,outlier_values.size)

In [ ]:
tdf['num_sold'].describe()

# 🔅 MODEL 

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
import lightgbm as lgb
import xgboost as xgb
import catboost as cb
from sklearn.model_selection import train_test_split , cross_val_score
from sklearn.preprocessing import LabelEncoder

In [ ]:

X=tdf.drop(['num_sold'],axis=1)
y=tdf["num_sold"]
X['week']=X['week'].astype('int')

In [ ]:
X_train , X_val ,y_train , y_val = train_test_split(X,y,test_size=0.24983 , random_state =47)

In [ ]:
def SMAPE(y_true,y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    smape=0
    if(len(y_true)==len(y_pred)):
        smape = (100/len(y_true)) * np.sum(2* np.abs(y_pred-y_true)/(np.abs(y_true)+np.abs(y_pred)))
    else:
        return
    return(smape)

In [ ]:
dic_model = {"RandomForest":RandomForestRegressor(n_estimators=100),
             'GradientBoosting':GradientBoostingRegressor(n_estimators=100),
            'XGradientBoosting':xgb.XGBRegressor(n_estimators=100),
            'CatBoostRegressor':cb.CatBoostRegressor(n_estimators=100),
            'LightGBM': lgb.LGBMRegressor(n_estimators=100)}

for i in dic_model:
    #Training
    print("Training with ",i+" Algorithm....")
    print()
    model = dic_model[i].fit(X_train,y_train)
    
    #Predicting
    print("Predicting with ",i+" Model....")
    print()
    prediction = model.predict(X_val)
    
    # Using SMAPE for predicting models
    print("SMAPE of ",i+" Model is ",SMAPE(y_val,prediction))
    print("------------------------------------------------------------------")

In [ ]:
# model.compile(tf.keras.optimizers.Adam(learning_rate=0.0001,beta_1=0.99),
#              loss='hinge',
#              metrics=['accuracy'])

In [ ]:
 # 'min_data_in_leaf': 186 'l2_leaf_reg': 0.018327921341491783,'rsm': 0.7531284161762606, 
params = { 'max_bin': 376, 
          'learning_rate': 1.299687, 'n_estimators': 4560, 'max_depth': 500, 
          'random_state': 47}
model_cat = lgb.LGBMRegressor(**params)
model_cat.fit(X,y,verbose=20)

In [ ]:
 prediction = model_cat.predict(X_val)
    
    # Using SMAPE for predicting models
print("SMAPE of ",i+" Model is ",SMAPE(y_val,prediction))
print("------------------------------------------------------------------")

In [ ]:
from collections import defaultdict
c= defaultdict(LabelEncoder)         # trick to do encoding and decoding vary easily 😁
 
x = Tdf[['country' ,'store','product' ,'year']]

x=x.apply(lambda x: c[x.name].fit_transform(x))
x.head()
Tdf[['country' ,'store','product' ,'year']]=x
Tdf

In [ ]:
X_test =Tdf.drop(['row_id','date'],axis=1)
X_test['week']=X_test['week'].astype('int')

In [ ]:
pred=model_cat.predict(X_test)

In [ ]:
pred.shape

In [ ]:
sub=pd.DataFrame()
sub["row_id"]=Tdf["row_id"]
sub["num_sold"]=np.ceil(pred)

In [ ]:
sub

In [ ]:
sub.to_csv('submission.csv' , index=False)